In [1]:
from wwl import wwl
from Utility import CollectInputNames, get_catkit_attribute
from Processdata import GraphBase, GraphGroup, CollectDatatoPandas
import os
from Analyzer import KF_validation
import time
import random
from GPR import GaussianProcessRegressor

In [2]:
database_path   = 'Data/small_adsorbate/'
feature_list = ['electron_affinity','vdw_radius']
graphs_object   = GraphGroup(database_path, cutoff_mult=1, weighted=False, skin=0.1)
file_names      = CollectInputNames(database_path)
graphs          = graphs_object.Collect_graph()
catkit_pool     = get_catkit_attribute()
node_attributes = graphs_object.Collect_node_attributes(feature_list, attributes_pool=catkit_pool, normalize=False)
ads_energies    = graphs_object.Collect_ads_energies()
datas           = CollectDatatoPandas(graphs, ads_energies, file_names)

In [3]:
kernel_matrix = wwl(graphs, node_features=node_attributes, num_iterations=1, sinkhorn=False, gamma=None)

Continuous node features provided, using CONTINUOUS propagation scheme.


In [4]:
kernel_matrix.shape

(312, 312)

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np
num_rows = kernel_matrix.shape[0]
indices = np.arange(num_rows)
train_idx, val_idx = train_test_split(indices, test_size=0.2, train_size=None)

generating the index of training and validation 

In [6]:
train_matrix = kernel_matrix[train_idx][:,train_idx]
test_matrix  = kernel_matrix[val_idx][:,train_idx]
kyy = kernel_matrix[val_idx][:,val_idx]
y_train = datas['target'][train_idx]; y_test = datas['target'][val_idx]

In [7]:
print(train_matrix.shape, test_matrix.shape, kyy.shape)

(249, 249) (63, 249) (63, 63)


In [8]:
gp=GaussianProcessRegressor(kernel='precomputed')

In [9]:
gp.fit(train_matrix, y_train)

GaussianProcessRegressor(kernel='precomputed')

In [10]:
test_pre, y_std = gp.predict(test_matrix, kyy=kyy, return_std=True)

In [11]:
y_std

array([0.01411262, 0.01783656, 0.01451263, 0.02299428, 0.01339926,
       0.01612978, 0.0137293 , 0.021597  , 0.01361209, 0.02563478,
       0.01488377, 0.01236459, 0.02218925, 0.05218045, 0.15874443,
       0.02080509, 0.15931354, 0.01474394, 0.01481799, 0.04678964,
       0.03661456, 0.01675487, 0.01463214, 0.12690989, 0.01967374,
       0.0243456 , 0.03242337, 0.01548856, 0.01848588, 0.05807736,
       0.01422667, 0.1270801 , 0.01634731, 0.01530949, 0.03534133,
       0.01751151, 0.02834901, 0.04651919, 0.01477396, 0.01465652,
       0.01699576, 0.02039873, 0.02109232, 0.02159805, 0.01490573,
       0.01819724, 0.02452122, 0.01629721, 0.01467638, 0.02930932,
       0.05220531, 0.03194591, 0.05948417, 0.01476598, 0.0133348 ,
       0.01421086, 0.0264905 , 0.01306746, 0.02621044, 0.08813617,
       0.01305638, 0.012352  , 0.01375289])

In [13]:
test_pre, cov = gp.predict(test_matrix, kyy=kyy, return_cov=True)

In [17]:
test_pre - y_test

81     0.468864
218    0.199800
105    0.058686
189   -0.409783
255    0.041590
211   -0.009383
163   -0.090291
308    0.229054
153    0.002347
123    0.640007
73     0.797608
22    -0.261935
118    0.164446
187    0.296769
229    1.703365
231    0.113125
292    1.582531
277    0.609322
310    0.798427
253   -0.380260
174   -0.382991
289    0.241304
138    0.218589
49    -0.148683
11    -0.925281
1      0.430662
14     0.260790
305    0.184344
102   -0.299188
139   -0.585356
         ...   
230    0.558277
282   -0.168840
151    0.449025
143    0.198562
124   -0.876360
39     0.233536
240   -0.000252
301    0.548894
297   -0.157039
21     0.251555
127    0.077407
95     2.263729
304   -0.190794
114    0.002845
30     0.431495
241    0.263191
209    0.643509
147    0.297511
269    0.001388
283    0.219191
126    0.184241
204    0.023283
79     0.000324
270   -0.134958
56    -0.008462
80     0.129240
12    -0.251669
46    -0.020688
248    0.282429
156    0.004917
Name: target, Length: 63